Задача: превратить дамп датасета Отзывуса (только тексты в виде csv файла) в пригодный для разметки csv файл.

Файл должен быть формата

```
text;аспект1;аспект2;аспект3;...
<Текст отзыва>;0|4;0|4;0|4;...
```

pd.NA - аспект не представлен ни в одном файле, 4 - представлен.<br>
1-3 зарезервированы для эмоциональной окраски

# Скачиваем дамп

In [20]:
import requests
from pathlib import Path
import os

In [21]:
DATASETS_DIRECTORY = Path("../data/datasets")
ANNOTATED_ALL_DIRECTORY = Path("../data/otzyvus-annotated/annotated_files_all")
ANNOTATED_REAL_DIRECTORY = Path("../data/otzyvus-annotated/annotated_files_real")
ANNOTATED_AI_DIRECTORY = Path("../data/otzyvus-annotated/annotated_files_ai")
DATASETS_DIRECTORY.mkdir(parents=True, exist_ok=True)
FEEDBACKS_CSV_URL = "https://drive.google.com/uc?export=download&id=1J1mcl3Dy_9B1DdKXdVDJGqpHll4DakQR"
FEEDBACKS_CSV_DIR = DATASETS_DIRECTORY / "feedbacks-2024-05-06.csv"

In [22]:
if not os.path.isfile(FEEDBACKS_CSV_DIR):
    response = requests.get(FEEDBACKS_CSV_URL)
    open(FEEDBACKS_CSV_DIR, "wb").write(response.content)

# Методы чистки текста и разбиения на предложения

In [23]:
import nltk

def clean_text(text: str):
    # Убираем ссылки
    clean = re.sub(u'(http|ftp|https):\/\/[^ ]+', '', text)
    # Убираем все неалфавитные символы кроме дефиса и апострофа
    clean = re.sub(u'[^а-я^А-Я^\w^\-^\']', ' ', clean)
    # Убираем тире
    clean = re.sub(u' - ', ' ', clean)
    # Убираем дубликаты пробелов
    clean = re.sub(u'\s+', ' ', clean)
    # Убираем пробелы в начале и в конце строки
    clean = clean.strip().lower()
    return clean

def get_sentences(text: str):
    # Токенизация
    return nltk.sent_tokenize(text, language="russian")

# Подготовка csv

In [24]:
# Получаем аспекты из названий файлов
labels = [f[:-4] for f in os.listdir(ANNOTATED_ALL_DIRECTORY) if os.path.isfile(ANNOTATED_ALL_DIRECTORY / f) and f[-4:] == ".txt"]

In [25]:
import pandas as pd

In [26]:
texts_df = pd.read_csv(FEEDBACKS_CSV_DIR, sep=";")
texts = texts_df["text"]
texts_len = len(texts)
# Итоговый словарь, где пока что на перес
annotation_dict = {"text": texts}
annotated_dict = {}
for label in labels:
    annotation_dict[label] = pd.Series([pd.NA] * texts_len)
    with open(ANNOTATED_ALL_DIRECTORY / (label+".txt"), "r") as file:
        annotated_dict[label] = file.read().split("\n")

In [28]:
for i, text in zip(range(texts_len), texts):
    for label in annotated_dict:
        intersection = set(get_sentences(text)) & set(annotated_dict[label])
        if intersection:
            annotation_dict[label][i] = 4
            for sentence in intersection:
                annotated_dict[label].remove(sentence)

In [29]:
result_df = pd.DataFrame(annotation_dict)
result_df = result_df[result_df["мусор"].isnull()].drop("мусор", axis=1)
result_df.to_csv(DATASETS_DIRECTORY / "ready_for_annotation.csv", sep=";", index=False)

In [30]:
for label in annotated_dict:
    with open(ANNOTATED_AI_DIRECTORY / (label+".txt"), "w") as file:
        file.write("\n".join(annotated_dict[label]))

In [31]:
for filename in os.listdir(ANNOTATED_ALL_DIRECTORY):
    with open(ANNOTATED_ALL_DIRECTORY / filename) as file:
        sentences_all = set(file.read().split("\n"))
    with open(ANNOTATED_AI_DIRECTORY / filename) as file:
        sentences_real = set(file.read().split("\n"))
    sentences_ai = sentences_all - sentences_real
    with open(ANNOTATED_REAL_DIRECTORY / filename, "w") as file:
        file.write("\n".join(sentences_ai))